In [1]:
# from libs.alibaba.visitor import Visitor
from libs.alibaba.alibaba import Alibaba
from libs.json import JSON
from openpyxl import load_workbook
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
from libs.CeleryTasks import tasks
import html
import time
import re
import os
import traceback
import types
import pendulum

In [2]:
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']

. storage markets.json
.\storage\markets.json was deserialized!


In [39]:
class Product_Ranking:
    api = 'https://hz-productposting.alibaba.com/product/ranksearch/rankSearch.htm'
    browser = None
    
    def __init__(self, market, headless=True, browser=None):
        self.market = market
        self.lid = market['lid']
        self.lpwd = market['lpwd']

        self.alibaba = None
        self.browser = browser
        self.headless = headless
        
    def login(self):
        self.alibaba = Alibaba(self.lid, self.lpwd, headless=self.headless, browser=self.browser)
        self.alibaba.login()
        self.browser = self.alibaba.browser
        
    def find_product_rank(self, kw, product_id):
        result = None
        input = self.browser.find_element_by_css_selector('#queryString')
        input.send_keys(Keys.CONTROL, 'a')
        input.send_keys(kw)
        input.send_keys(Keys.RETURN)

        css_table = '#rank-searech-table'
        table = WebDriverWait(self.browser, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_table))
        )
        idx = 0
        for tr in table.find_elements_by_css_selector('tbody tr'):
            if '无匹配结果' in tr.text:
                return None
            a = tr.find_element_by_css_selector('td.products a')
            if product_id in a.get_attribute('href'):
                a = tr.find_element_by_css_selector('td.ranking a')
                url = a.get_attribute('href')
                text = a.text
                print(text, end='\t')
                page = re.search('约第(\d+)页，第(\d+)位', text).group(1)
                pos = re.search('约第(\d+)页，第(\d+)位', text).group(2)
                result = {'keyword':kw, 'url':url, 'index':idx, 'page':page, 'position':pos}
                break
            idx += 1
        if result is None:
            result = {'index': idx, 'keyword':kw, 'page':'', 'position':''}
        return result
    
    def fill_excel_file(self, file):
        wb = load_workbook(file)
        sheet = wb.worksheets[0]
        pid = str(sheet.cell(row=1,column=5).value)
        dt_cell = sheet.cell(row=2, column=5)
        dt_cell.value = pendulum.now().to_datetime_string()
        for i in range(7, 9999):
            cell = sheet.cell(row=i,column=4)
            if cell.value is None:
                break
            kw = ' '.join(cell.value.strip().split('\xa0'))
            print(sheet.cell(row=i,column=1).value, end='\t')
            print(sheet.cell(row=i,column=2).value, end='\t')
            print(sheet.cell(row=i,column=3).value, end='\t')
            print(kw, end='\t')
            ranking = self.find_product_rank(kw, pid)
            if ranking is None:
                result = 'X'
            elif ranking['page'] and ranking['position']:
                result = str(ranking['index']) + '-' + ranking['page'] + '-' + ranking['position']
            else:
                result = str(ranking['index'])
            cell = sheet.cell(row=i,column=5)
            cell.value = result
            print(result)
        wb.save(file)
        print('done!')
        
    def find_keyword_ranking(self, kw):

        result = []
        input = self.browser.find_element_by_css_selector('#queryString')
        input.send_keys(Keys.CONTROL, 'a')
        input.send_keys(kw)
        input.send_keys(Keys.RETURN)

        css_table = '#rank-searech-table'
        table = WebDriverWait(self.browser, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, css_table))
        )

        idx = 0
        for tr in table.find_elements_by_css_selector('tbody tr'):
            if '无匹配结果' in tr.text:
                break
            a = tr.find_element_by_css_selector('td.products a')
            url = a.get_attribute('href')
            ali_id = re.search('\?id=(\d+)', url).group(1)
            a = tr.find_element_by_css_selector('td.ranking a')
            text = a.text
            page = re.search('约第(\d+)页，第(\d+)位', text).group(1)
            pos = re.search('约第(\d+)页，第(\d+)位', text).group(2)

            result.append({'keyword':kw, 'url':url, 'ali_id': ali_id, 'index':idx, 'page':page, 'position': pos})
            idx += 1
        return result

In [ ]:
def fill_excel_file(self, file):
    wb = load_workbook(file)
    sheet = wb.worksheets[0]
    pid = str(sheet.cell(row=1,column=5).value)
    dt_cell = sheet.cell(row=2, column=5)
    dt_cell.value = pendulum.now().to_datetime_string()
    for i in range(7, 9999):
        cell = sheet.cell(row=i,column=4)
        if cell.value is None:
            break
        kw = ' '.join(cell.value.strip().split('\xa0'))
        print(sheet.cell(row=i,column=1).value, end='\t')
        print(sheet.cell(row=i,column=2).value, end='\t')
        print(sheet.cell(row=i,column=3).value, end='\t')
        print(kw, end='\t')
        ranking = self.find_product_rank(kw, pid)
        if ranking:
            result = ranking['page'] + '-' + ranking['position']
        else:
            result = 'X'
        cell = sheet.cell(row=i,column=5)
        cell.value = result
        print(result)
    wb.save(file)
    print('done!')

pr.fill_excel_file = types.MethodType(fill_excel_file, pr)

In [ ]:
pr.find_keyword_ranking = types.MethodType(find_keyword_ranking, pr)

In [40]:
alibaba = pr.alibaba
browser = pr.browser
pr = Product_Ranking(market, headless=False)
pr.alibaba = alibaba
pr.browser = browser

In [4]:
pr = Product_Ranking(market, headless=False)

In [5]:
pr.login()

primary 打开网址：https://i.alibaba.com
primary 等待登陆页面加载 ... ...
primary 输入登录信息， 并登录
danger 登录 阿里巴巴 国际站 失败! Message: 



Traceback (most recent call last):
  File "E:\workspace\python\Market Management\libs\alibaba\alibaba.py", line 131, in login
    EC.presence_of_element_located((By.CSS_SELECTOR, 'header div[data-role="user"]')))
  File "e:\worksp~1\python\market~1\venv\lib\site-packages\selenium\webdriver\support\wait.py", line 80, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 



In [7]:
pr.browser.get(pr.api)

In [ ]:
#pr.find_keyword_ranking('mink eyelash')

In [ ]:
# pr.find_product_rank(kw, pid)

In [49]:
files = []
for i in range(50):
    files.append('')

files[0] = 'product_ranking_mink_eyelash.xlsx'
files[1] = 'product_ranking_magnetic_eyelash.xlsx'
files[2] = 'product_ranking_Easy_Fan_Lashes.xlsx'
files[3] = 'product_ranking_mink_eyelash_extension.xlsx'
files[4] = 'product_ranking_individual_eyelash_extension.xlsx'
files[5] = 'product_ranking_brown_eyelash_extension.xlsx'
files[6] = 'product_ranking_flat_eyelash_extension.xlsx'
files[7] = 'product_ranking_colorful_eyelash_extension.xlsx'
files[8] = 'product_ranking_blooming_eyelash_extension.xlsx'
files[9] = 'product_ranking_camellia_eyelash_extension.xlsx'
files[10] = 'product_ranking_ombre_eyelash_extension.xlsx'
files[11] = 'product_ranking_flat_split_tips_eyelash_extension.xlsx'
files[12] = 'product_ranking_premade_fans_eyelash_extension.xlsx'
files[13] = 'product_ranking_premade_fans_eyelash_extension_01_60762376749.xlsx'
files[14] = 'product_ranking_premade_fans_eyelash_extension_02_60795868865.xlsx'
files[15] = 'product_ranking_premade_fans_eyelash_extension_03_60761678736.xlsx'

In [54]:
file = files[15]
pr.fill_excel_file(file)

230	51	539	volume lash	2
25	4	367	pre made volume fan	约第2页，第11位	0-2-11
41	7	338	pre made fan	5
45	3	196	fan lash	4
8	0	194	10 eyelash	2
60	3	172	3d volume eyelash extension	2
19	0	167	eyelash extension pre made fans	约第2页，第5位	0-2-5
23	1	163	russian eyelash	4
26	1	155	mink volume lash	2
26	1	155	4d lash	1
13	0	150	4d eyelash	1
12	0	145	russian volume mink eyelash extension	约第10页，第2位	2-10-2
53	8	141	russian volume eyelash	3
5	0	138	10d lash	3
7	0	136	eyelash extension 6d	3
1	0	132	lash 5d	约第16页，第13位	2-16-13
29	2	124	5d eyelash	3
2	0	122	eyelash extension pre made	约第2页，第3位	0-2-3
5	2	122	10d eyelash	4
37	3	122	fanned lash	4
15	0	118	3d premade volume fan	4
5	1	116	eyelash extension 10d	4
27	0	114	pre made fan lash	5
3	0	112	eyelash 5d	3
3	0	112	eyelash russian	3
78	3	112	3d lash extension	1
0	0	110	lash russian	4
15	0	107	mini fan eyelash	X
10	0	105	volume 3d 0.07 korea mink eyelash extension	X
15	0	105	eyelash 6d	3
2	0	105	hybrid lash	X
7	0	102	eyelash 3d extension	2
4	0	100	korean volume 

In [ ]:
product_ranking_overview

In [ ]:
%store -r sorted_kws

In [ ]:
products = pr.alibaba.get_posted_product_info(999)

In [ ]:
ps = {}
for p in products:
    ps[p['ali_id']] = p

In [ ]:
file = 'products ranking overview.xlsx'
wb = load_workbook(file)
sheet = wb.worksheets[0]
for i in range(1,999):
    kw = sheet.cell(row=i,column=2).value
    if kw is None:
        break
    print(sheet.cell(row=i,column=1).value, kw, end="\t")
    while True:
        try:
            result = pr.find_keyword_ranking(kw)
            break
        except TimeoutException:
            pr.browser.get(pr.api)
            continue
    if result:
        fp = result[0]
        p = ps[fp['ali_id']]
        sheet.cell(row=i, column=3).value = fp['page']
        sheet.cell(row=i, column=4).value = fp['position']
        sheet.cell(row=i, column=5).value = fp['ali_id']
        sheet.cell(row=i, column=6).value = p['pid']
        sheet.cell(row=i, column=7).value = p['title']
    else:
        sheet.cell(row=i, column=3).value = ' '
        sheet.cell(row=i, column=4).value = ' '
        sheet.cell(row=i, column=5).value = ' '
        sheet.cell(row=i, column=6).value = ' '
        sheet.cell(row=i, column=7).value = ' '
        
    print(fp['page'], fp['position'], fp['ali_id'], p['pid'], p['title'])
#     break
wb.save(file)

In [ ]:
Wangqingxia25160

In [6]:
market

{'name': 'Eyelashes',
 'directory': 'E:\\glitter\\Eyelashes',
 'lid': 'eyelash@glittergroupcn.com',
 'lpwd': 'Wangzhen68970747',
 'homepage': 'https://glittereyelash.en.alibaba.com',
 'lname': 'Jeremy Wang',
 'mobile': '17685593100',
 'accounts': [{'lid': 'eyelash1@glittergroupcn.com',
   'lpwd': 'Zhangying25160',
   'lname': 'Emily Zhang',
   'mobile': '17685593100',
   'disabled': False},
  {'lid': 'eyelash2@glittergroupcn.com',
   'lpwd': 'Huyinping18130',
   'lname': 'Ada Alva',
   'mobile': '18563918130',
   'disabled': False}]}